In [ ]:
setwd("/projects/PPC/analysis/ppc_eqtls")

source("scripts/packages.R"  )
source("scripts/functions.R" )

dir.create("pipeline/1.2.expression"            , showWarnings = F)
dir.create("pipeline/1.2.expression/tpm_gene"   , showWarnings = F)
dir.create("pipeline/1.2.expression/tpm_isoform", showWarnings = F)
dir.create("pipeline/1.2.expression/use_isoform", showWarnings = F)

# Input files

metadata              =              fread("pipeline/1.1.metadata/metadata.txt"       , sep = "\t", header = T, data.table = F)
meta_rna              =              fread("pipeline/1.1.metadata/meta_rna.txt"       , sep = "\t", header = T, data.table = F)
meta_subject          =              fread("pipeline/1.1.metadata/meta_subject.txt"   , sep = "\t", header = T, data.table = F)
flagstat              = add_rownames(fread("input/phenotypes/flagstat.txt"            , sep = "\t", header = T, data.table = F))
gene_tpm              = add_rownames(fread("input/phenotypes/gene_tpm.txt"            , sep = "\t", header = T, data.table = F))
isoform_percent_use   = add_rownames(fread("input/phenotypes/isoform_percent_use.txt" , sep = "\t", header = T, data.table = F))
isoform_tpm           = add_rownames(fread("input/phenotypes/isoform_tpm.txt"         , sep = "\t", header = T, data.table = F))

# Get expression matrices and filter metadata files

flagstat                = flagstat           [ meta_rna$rna_id,]
gene_tpm                = gene_tpm           [,meta_rna$rna_id ]
isoform_percent_use     = isoform_percent_use[,meta_rna$rna_id ]
isoform_tpm             = isoform_tpm        [,meta_rna$rna_id ]

message(paste("# samples in flagstat:"    , nrow(flagstat           )))
message(paste("# samples in gene_tpm:"    , ncol(gene_tpm           )))
message(paste("# samples in iso_pct_use:" , ncol(isoform_percent_use)))
message(paste("# samples in iso_tpm:"     , ncol(isoform_tpm        )))

# Create RNA covariates (from flagstat)

covariates_rna = data.frame(row.names   = rownames(flagstat),
                            total_reads = flagstat$total_reads,
                            uniquely_mapped_reads                         = 100 * flagstat$both_mapped / flagstat$total_reads,
                            uniquely_mapped_reads_to_canonical_chromsomes = 100 * rowSums(flagstat[, c("autosomal_reads", "chrX_reads", "chrY_reads")]) / flagstat$total_reads,
                            mitochondrial_reads                           = 100 * flagstat$chrM_reads / flagstat$total_reads)


In [12]:
# Find expressed genes
gene_info    = fread("input/phenotypes/gene_info.txt"   , sep = "\t", header = T, data.table = F)
isoform_info = fread("input/phenotypes/isoform_info.txt", sep = "\t", header = T, data.table = F)

gene_info$transcript_id = gene_info$gene_id

# Computer PEER with only 2000 genes

In [ ]:
divide_phenotypes_tissue("pipeline/1.2.expression/tpm_gene", gene_tpm, run_peer = T, normalize = T, gene_ids = NULL, filter_exp = T, phenotype_min_value = 1, phenotype_min_samples = 0.1, peer_factor_n = 100, n_genes = 2000)


# Compute PEER with only 5000 genes

In [ ]:
divide_phenotypes_tissue("pipeline/1.2.expression/tpm_gene", gene_tpm, run_peer = T, normalize = T, gene_ids = NULL, filter_exp = T, phenotype_min_value = 1, phenotype_min_samples = 0.1, peer_factor_n = 60, n_genes = 5000)


---------------------------

Normalizing expression...

Total genes/peaks = 62492

Total samples = 107

Total expressed genes = 17098

Computing PEER...



In [ ]:
message("Done.")

In [24]:
# Run individual gene
divide_phenotypes_by_gene = function(expdata, outfolder)
{
    message("Dividing phenotype data by gene/peak...")
    
    dir.create(outfolder, showWarnings = FALSE)

    gene_ids   = expdata$gene_ids
    sample_ids = expdata$sample_ids
    rawexp     = expdata$tpm_f
    normexp    = expdata$tpm_f_std_norm
    
    invisible(lapply(gene_ids, function(gene)
    {
        outdata = data.frame(sample_id = sample_ids, raw = as.numeric(rawexp[gene,sample_ids]), norm = as.numeric(normexp[gene,sample_ids]))
        fwrite(outdata, paste(outfolder, paste(gene, "txt", sep = "."), sep = "/"), sep = "\t", row.names = FALSE, col.names = TRUE)
    }))
}

meta_rna              =              fread("pipeline/1.1.metadata/meta_rna.txt"       , sep = "\t", header = T, data.table = F)
gene_tpm              = add_rownames(fread("input/phenotypes/gene_tpm.txt"            , sep = "\t", header = T, data.table = F))

colnames(gene_tpm)[which(colnames(gene_tpm) == "S02307_PPC_C6P20_PPC083_RNA_R02L01S01")] = "S02307_PPC_C6P20_PPC134_RNA_R02L01S01"
colnames(gene_tpm)[which(colnames(gene_tpm) == "T600_PPC_C4P19_PPC083_RNA_R02L01S01")] = "T600_PPC_C4P19_PPC029_RNA_R02L01S01"

expdata = normalize_tpm("pipeline/1.2.expression/tpm_gene", gene_tpm, gene_info, normalize = T, gene_ids = NULL, filter_exp = T, min_tpm = 1, min_samples = 0.1, use = F)

gene_ids   = expdata$gene_ids
sample_ids = expdata$sample_ids
rawexp     = expdata$tpm_f
normexp    = expdata$tpm_f_std_norm

for (gene in c('ENSG00000006327','ENSG00000168286','ENSG00000104899','ENSG00000105642','ENSG00000287069','ENSG00000013561'))
{
    outdata = data.frame(sample_id = sample_ids, raw = as.numeric(rawexp[gene,sample_ids]), norm = as.numeric(normexp[gene,sample_ids]))
    fwrite(outdata, paste("pipeline/1.2.expression/tpm_gene", paste(gene_info[gene_info$gene_id %like% gene,]$gene_id, "txt", sep = "."), sep = "/"), sep = "\t", row.names = FALSE, col.names = TRUE)
}

# divide_phenotypes_by_gene(expdata, "pipeline/1.2.expression/tpm_gene")


Normalizing expression...

Total genes/peaks = 62492

Total samples = 107

Total expressed genes = 17098



# Prepare Isoform TPM

In [7]:
genes_expressed        = readLines("pipeline/1.2.expression/tpm_gene.gene_ids.txt")
isoforms_expressed     = isoform_info[isoform_info$gene_id %in% genes_expressed,]
gene2isoform_expressed = table(isoforms_expressed$gene_id)
isoforms_to_remove     = names(gene2isoform_expressed[gene2isoform_expressed == 1])
isoforms_expressed     = isoforms_expressed[!isoforms_expressed$gene_id %in% isoforms_to_remove, "transcript_id"]


In [3]:
divide_phenotypes_tissue("pipeline/1.2.expression/tpm_isoform", isoform_tpm, run_peer = F, 
                         normalize = T, gene_ids = isoforms_expressed, filter_exp = T, phenotype_min_value = 1, phenotype_min_samples = 0.1)


---------------------------

Normalizing expression...

Total genes/peaks = 142818

Total samples = 107

Total expressed genes = 61144

Dividing phenotype data by gene/peak...



# Prepare Isoform use

In [5]:
isoforms_expressed = readLines("pipeline/1.2.expression/tpm_isoform.gene_ids.txt")

divide_phenotypes_tissue("pipeline/1.2.expression/use_isoform", isoform_percent_use, geneinfo = isoform_info, run_peer = FALSE, normalize = TRUE, gene_ids = isoforms_expressed, 
                         filter_exp = TRUE, phenotype_min_value = 10, phenotype_min_samples = 0.1, use = TRUE)



---------------------------

Normalizing expression...

Total genes/peaks = 61144

Total samples = 107

Total expressed genes = 30925

Dividing phenotype data by gene/peak...



# Update metadata and covariates

In [ ]:
isoforms_use_expressed = readLines("pipeline/1.2.expression/use_isoform.gene_ids.txt")

# Rewrite gene and isoform information
gene_info_filtered    = gene_info   [gene_info   $gene_id       %in% genes_expressed       ,]
isoform_info_filtered = isoform_info[isoform_info$transcript_id %in% isoforms_use_expressed,]

fwrite(gene_info_filtered   , "pipeline/1.2.expression/gene_info.txt"    , sep = "\t", col.names = TRUE, row.names = FALSE)
fwrite(isoform_info_filtered, "pipeline/1.2.expression/isoform_info.txt" , sep = "\t", col.names = TRUE, row.names = FALSE)

# Write metadata output
peer_factors   = add_rownames(fread("pipeline/1.2.expression/tpm_gene.peer.factors.txt", sep = "\t", header = TRUE , data.table = FALSE))
covariates_rna = add_rownames(merge(peer_factors, covariates_rna, by = "row.names"))

head(covariates_rna)

pc_factors     = add_rownames(fread("pipeline/1.2.expression/tpm_gene.pc.factors.txt", sep = "\t", header = T, data.table = F))
covariates_rna = add_rownames(merge(covariates_rna, pc_factors, by = "row.names"))

head(covariates_rna)

wgs_id_vcf = unlist(strsplit(system("bcftools query -l input/genotype/ipscore.vcf.gz", intern = TRUE), split = "\t"))
metadata$vcf       = FALSE

metadata[metadata$wgs_id %in% wgs_id_vcf, "vcf"] = TRUE

fwrite(covariates_rna, "pipeline/1.2.expression/covariates_rna.txt" , sep = "\t", col.names = TRUE, row.names = TRUE )
fwrite(metadata      , "pipeline/1.2.expression/metadata.txt"       , sep = "\t", col.names = TRUE, row.names = FALSE)
fwrite(meta_subject  , "pipeline/1.2.expression/meta_subject.txt"   , sep = "\t", col.names = TRUE, row.names = FALSE)